# Courier Charges Audit Project

End-to-end reconciliation of Company X orders vs Courier invoices.

This notebook:
- Loads all datasets
- Calculates weights & slabs
- Maps zones
- Applies rate cards
- Computes expected vs billed
- Produces summary
- Exports final Excel (2 sheets)


In [ ]:

import pandas as pd
import numpy as np


## 1. Load Datasets (keep all Excel files in same folder as this notebook)

In [ ]:

order_report = pd.read_excel("Company X - Order Report.xlsx")
sku_master = pd.read_excel("Company X - SKU Master.xlsx")
pincode_zones = pd.read_excel("Company X - Pincode Zones.xlsx")
invoice = pd.read_excel("Courier Company - Invoice.xlsx")
rates = pd.read_excel("Courier Company - Rates.xlsx")

print(order_report.shape, sku_master.shape, pincode_zones.shape, invoice.shape, rates.shape)


## 2. Calculate Total Weight per Order from SKU Master

In [ ]:

df = order_report.merge(sku_master, on="SKU", how="left")
df["total_g"] = df["Order Qty"] * df["Weight (g)"]

order_weights = df.groupby("ExternOrderNo", as_index=False)["total_g"].sum()
order_weights["Total weight as per X (KG)"] = order_weights["total_g"] / 1000

def slab(w):
    return np.ceil(w/0.5)*0.5

order_weights["Weight slab as per X (KG)"] = order_weights["Total weight as per X (KG)"].apply(slab)

order_weights.head()


## 3. Merge with Courier Invoice & Compute Courier Slabs

In [ ]:

inv = invoice.rename(columns={"Order ID":"ExternOrderNo"})
merged = inv.merge(order_weights[["ExternOrderNo","Total weight as per X (KG)","Weight slab as per X (KG)"]],
                   on="ExternOrderNo", how="left")

merged["Weight slab charged by Courier Company (KG)"] = merged["Charged Weight"].apply(slab)
merged.head()


## 4. Map Delivery Zones Using Pincodes

In [ ]:

zones = pincode_zones.rename(columns={"Zone":"Delivery Zone as per X"})
merged = merged.merge(zones, on=["Warehouse Pincode","Customer Pincode"], how="left")
merged.rename(columns={"Zone":"Delivery Zone charged by Courier Company"}, inplace=True)

merged[["ExternOrderNo","Delivery Zone as per X"]].head()


## 5. Apply Rate Card to Calculate Expected Charges

In [ ]:

rate = rates.iloc[0]

def calc_expected(row):
    zone = row["Delivery Zone as per X"]
    slab_w = row["Weight slab as per X (KG)"]
    add_units = max(0, (slab_w-0.5)/0.5)

    f_fixed = rate[f"fwd_{zone}_fixed"]
    f_add = rate[f"fwd_{zone}_additional"] * add_units
    total = f_fixed + f_add

    if "rto" in str(row["Type of Shipment"]).lower():
        total += rate[f"rto_{zone}_fixed"] + rate[f"rto_{zone}_additional"] * add_units

    return round(total,2)

merged["Expected Charge as per X (Rs.)"] = merged.apply(calc_expected, axis=1)
merged["Difference Between Expected Charges and Billed Charges (Rs.)"] = merged["Expected Charge as per X (Rs.)"] - merged["Billing Amount (Rs.)"]

merged.head()


## 6. Final Order-Level Output

In [ ]:

final = merged.rename(columns={
    "ExternOrderNo":"Order ID",
    "AWB Code":"AWB Number",
    "Charged Weight":"Total weight as per Courier Company (KG)",
    "Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.)"
})

final = final[[
    "Order ID","AWB Number",
    "Total weight as per X (KG)","Weight slab as per X (KG)",
    "Total weight as per Courier Company (KG)","Weight slab charged by Courier Company (KG)",
    "Delivery Zone as per X","Delivery Zone charged by Courier Company",
    "Expected Charge as per X (Rs.)","Charges Billed by Courier Company (Rs.)",
    "Difference Between Expected Charges and Billed Charges (Rs.)"
]]

final.head()


## 7. Summary Table (Correct / Over / Under Charged)

In [ ]:

summary = pd.DataFrame({
    "Category":[
        "Correctly Charged",
        "Overcharged",
        "Undercharged"
    ],
    "Count":[
        (final["Difference Between Expected Charges and Billed Charges (Rs.)"]==0).sum(),
        (final["Difference Between Expected Charges and Billed Charges (Rs.)"]>0).sum(),
        (final["Difference Between Expected Charges and Billed Charges (Rs.)"]<0).sum()
    ],
    "Amount (Rs.)":[
        final.loc[final["Difference Between Expected Charges and Billed Charges (Rs.)"]==0,
                  "Charges Billed by Courier Company (Rs.)"].sum(),
        final.loc[final["Difference Between Expected Charges and Billed Charges (Rs.)"]>0,
                  "Difference Between Expected Charges and Billed Charges (Rs.)"].sum(),
        final.loc[final["Difference Between Expected Charges and Billed Charges (Rs.)"]<0,
                  "Difference Between Expected Charges and Billed Charges (Rs.)"].sum()
    ]
})

summary


## 8. Export Final Excel (2 Sheets)

In [ ]:

with pd.ExcelWriter("Cointab_Final_Output.xlsx", engine="xlsxwriter") as writer:
    final.to_excel(writer, sheet_name="Order_Level", index=False)
    summary.to_excel(writer, sheet_name="Summary", index=False)

print("Exported: Cointab_Final_Output.xlsx")
